In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException,NoSuchElementException  # Import TimeoutException
import pandas as pd
import undetected_chromedriver as uc
import numpy as np
import os
from datetime import datetime
import random
from time import sleep
import time

In [2]:
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang web bán hàng\Cellphones\Link_product_cellphones.csv")
list_links = df['links'].to_list()
list_links

['https://cellphones.com.vn/iphone-15.html',
 'https://cellphones.com.vn/iphone-15-256gb.html',
 'https://cellphones.com.vn/iphone-15-512gb.html',
 'https://cellphones.com.vn/iphone-15-plus.html',
 'https://cellphones.com.vn/iphone-15-plus-256gb.html',
 'https://cellphones.com.vn/iphone-15-plus-512gb.html',
 'https://cellphones.com.vn/iphone-15-pro.html',
 'https://cellphones.com.vn/iphone-15-pro-256gb.html',
 'https://cellphones.com.vn/iphone-15-pro-512gb.html',
 'https://cellphones.com.vn/iphone-15-pro-1tb.html',
 'https://cellphones.com.vn/iphone-15-pro-max.html',
 'https://cellphones.com.vn/iphone-15-pro-max-512gb.html',
 'https://cellphones.com.vn/iphone-15-pro-max-1tb.html',
 'https://cellphones.com.vn/samsung-galaxy-s23-fe.html',
 'https://cellphones.com.vn/samsung-galaxy-s23-128gb.html',
 'https://cellphones.com.vn/samsung-galaxy-s23.html',
 'https://cellphones.com.vn/samsung-galaxy-s23-plus.html',
 'https://cellphones.com.vn/samsung-galaxy-s23-plus-512gb.html',
 'https://cellp

## Get Infooo

In [3]:
def safe_find_text_with_fallback(driver, primary_selector, fallback_selector):
    try:
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, primary_selector)))
        return element.text
    except TimeoutException:
        try:
            element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, fallback_selector)))
            return element.text
        except TimeoutException:
            return None

def safe_find_text(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((by, selector)))
        return element.text
    except TimeoutException:
        return None
    
def get_comment(driver, link):
    products = []  # Danh sách để lưu trữ thông tin sản phẩm
    driver.get(link)
    time.sleep(5)  # Đợi trang tải và cookies có hiệu lực

    print(link)

    try:
        current_date = datetime.now().strftime("%Y-%m-%d")
        print(current_date)
        
        product_title = safe_find_text(driver, "#productDetailV2 > section > div.box-header.is-flex.is-align-items-center.box-header-desktop > div.box-product-name > h1")
        print(product_title)
        current_price = safe_find_text_with_fallback(driver, 
                                                      "#trade-price-tabs > div > div > div.tpt-box.has-text-centered.is-flex.is-flex-direction-column.is-flex-wrap-wrap.is-justify-content-center.is-align-items-center.active > p.tpt---price",
                                                      "#productDetailV2 > section > div.box-detail-product.columns.m-0 > div.box-detail-product__box-center.column.bannerTopHead > div.block-box-price > div > p.product__price--through")
        print(current_price)
        discounted_price = safe_find_text_with_fallback(driver,
                                                         "#trade-price-tabs > div > div > div.tpt-box.has-text-centered.is-flex.is-flex-direction-column.is-flex-wrap-wrap.is-justify-content-center.is-align-items-center.active > p.tpt---sale-price",
                                                         "#productDetailV2 > section > div.box-detail-product.columns.m-0 > div.box-detail-product__box-center.column.bannerTopHead > div.block-box-price > div > p.product__price--show")
        print(discounted_price)
        rating_average = safe_find_text(driver, "#productDetailV2 > section > div.block-content-product > div.block-content-product-left > div.boxReview > div.boxReview-review.is-flex > div.boxReview-score.is-flex.is-justify-content-center.is-align-items-center.has-product > p.title.is-4.m-0.p-0")
        rating_count = safe_find_text(driver, "#productDetailV2 > section > div.block-content-product > div.block-content-product-left > div.boxReview > div.boxReview-review.is-flex > div.boxReview-score.is-flex.is-justify-content-center.is-align-items-center.has-product > p.boxReview-score__count > strong")
        print(rating_count)
        product_info = {
            'Platform': 'CellPhones',
            'URL': link,
            'Date Crawled': current_date,
            'Product Title': product_title,
            'Current Price': current_price,
            'Discounted Price': discounted_price,
            'Rating Average': rating_average,
            'Rating Count': rating_count,
        }
        
        products.append(product_info)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        return products


In [4]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

products = []
for link in list_links:
    products.extend(get_comment(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
product_df = pd.DataFrame(products)

https://cellphones.com.vn/iphone-15.html
2024-04-20
iPhone 15 128GB | Chính hãng VN/A
22.990.000đ
19.290.000đ
9
https://cellphones.com.vn/iphone-15-256gb.html
2024-04-20
iPhone 15 256GB | Chính hãng VN/A
25.990.000đ
23.490.000đ
None
https://cellphones.com.vn/iphone-15-512gb.html
2024-04-20
iPhone 15 512GB | Chính hãng VN/A
31.990.000₫
28.490.000₫
None
https://cellphones.com.vn/iphone-15-plus.html
2024-04-20
iPhone 15 Plus 128GB | Chính hãng VN/A
25.990.000đ
22.490.000đ
6
https://cellphones.com.vn/iphone-15-plus-256gb.html
2024-04-20
iPhone 15 Plus 256GB | Chính hãng VN/A
28.990.000đ
25.690.000đ
None
https://cellphones.com.vn/iphone-15-plus-512gb.html
2024-04-20
iPhone 15 Plus 512GB | Chính hãng VN/A
34.990.000₫
32.490.000₫
1
https://cellphones.com.vn/iphone-15-pro.html
2024-04-20
iPhone 15 Pro 128GB | Chính hãng VN/A
28.990.000₫
24.990.000₫
None
https://cellphones.com.vn/iphone-15-pro-256gb.html
2024-04-20
iPhone 15 Pro 256GB | Chính hãng VN/A
31.990.000₫
27.690.000₫
None
https://cellp

In [5]:
product_df

,Platform,URL,Date Crawled,Product Title,Current Price,Discounted Price,Rating Average,Rating Count
0,CellPhones,https://cellphones.com.vn/iphone-15.html,2024-04-20,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.290.000đ,4.9/5,9
1,CellPhones,https://cellphones.com.vn/iphone-15-256gb.html,2024-04-20,iPhone 15 256GB | Chính hãng VN/A,25.990.000đ,23.490.000đ,None,None
2,CellPhones,https://cellphones.com.vn/iphone-15-512gb.html,2024-04-20,iPhone 15 512GB | Chính hãng VN/A,31.990.000₫,28.490.000₫,None,None
3,CellPhones,https://cellphones.com.vn/iphone-15-plus.html,2024-04-20,iPhone 15 Plus 128GB | Chính hãng VN/A,25.990.000đ,22.490.000đ,5.0/5,6
4,CellPhones,https://cellphones.com.vn/iphone-15-plus-256gb...,2024-04-20,iPhone 15 Plus 256GB | Chính hãng VN/A,28.990.000đ,25.690.000đ,None,None
5,CellPhones,https://cellphones.com.vn/iphone-15-plus-512gb...,2024-04-20,iPhone 15 Plus 512GB | Chính hãng VN/A,34.990.000₫,32.490.000₫,5.0/5,1
6,CellPhones,https://cellphones.com.vn/iphone-15-pro.html,2024-04-20,iPhone 15 Pro 128GB | Chính hãng VN/A,28.990.000₫,24.990.000₫,None,None
7,CellPhones,https://cellphones.com.vn/iphone-15-pro-256gb....,2024-04-20,iPhone 15 Pro 256GB | Chính hãng VN/A,31.990.000₫,27.690.000₫,None,None
8,CellPhones,https://cellphones.com.vn/iphone-15-pro-512gb....,2024-04-20,iPhone 15 Pro 512GB | Chính hãng VN/A,37.990.000₫,34.690.000₫,None,None
9,CellPhones,https://cellphones.com.vn/iphone-15-pro-1tb.html,2024-04-20,iPhone 15 Pro 1TB | Chính hãng VN/A,43.990.000₫,39.490.000₫,None,None


In [6]:
product_df.to_csv('product_smartphone_CellPhones.csv', mode='a', header=not os.path.exists('product_smartphone_Cellphones.csv'), index=False, encoding='utf-8-sig')

## Get Comment

In [17]:
def get_commentt(driver, link):
    comments_list = []  # Danh sách để lưu trữ các comment
    review_link = link.replace('.html', '') + '/review'
    driver.get(review_link)
    sleep(10)  # Đợi trang tải và cookies có hiệu lực
    print(review_link)

    # Đợi và cuộn đến phần tổng quan bình luận
    try:
        comment_overview = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#review')))
        driver.execute_script("arguments[0].scrollIntoView(true);", comment_overview)
        sleep(1)
    except Exception as e:
        print(f"Không thể tìm thấy hoặc cuộn đến bình luận. Lỗi: {e}")
        return comments_list  # Trả về danh sách trống nếu không tìm thấy phần tổng quan bình luận

    # Cố gắng click vào nút "Xem thêm" nếu có
    try:
        xemthem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[3]/div[2]/div[2]/div[7]/a/div')  # Cập nhật XPath cho nút Xem thêm
        xemthem.click()
        sleep(5)  # Đợi cho đến khi nội dung mới được tải
    except Exception as e:
        print("Không thể click vào nút Xem thêm. Tiếp tục lấy dữ liệu bình luận.")

    while True:
        try:
            # Chú ý: Bạn cần chỉnh sửa các selector cho phù hợp với cấu trúc HTML của trang bạn muốn crawl
            customer_names = safe_find_elements(driver, '.name')
            time_comments = safe_find_elements(driver, '.date-time')
            contents = safe_find_elements(driver, ".comment-content")
            user = safe_find_elements(driver, "/html/body/div[1]/div/div/div[3]/div[2]/div[2]/div[7]/div[2]/div[1]/div/div/span", by=By.XPATH)
            content_rate = safe_find_elements(driver, ".boxReview-comment-item-review")
            for i in range(len(customer_names)):
                comment_info = {
                    'Platform': 'CellPhones',
                    'URL': link,
                    'Customer Name': customer_names[i],
                    'Time Comment': time_comments[i] if i < len(time_comments) else None,
                    'Content': contents[i] if i < len(contents) else None,
                    'status': user[i] if i < len(user) else None,
                    'content_rate': content_rate[i] if i < len(content_rate) else None,
                }
                comments_list.append(comment_info)
                print(comments_list)
            try:
                # Tìm element có text › để click
                next_page = driver.find_element(By.XPATH, '//a[contains(text(), "›")]')
                driver.execute_script("arguments[0].click();", next_page)
                sleep(5)  # Đợi trang mới tải xong
                print("next_page")
            except NoSuchElementException:
                break  # Thoát khỏi vòng lặp nếu không tìm thấy nút Next page
        except TimeoutException:
            print("Không tìm thấy bình luận cho link này. Chuyển sang link tiếp theo.")
            break

    return comments_list

def safe_find_elements(driver, selector, by=By.CSS_SELECTOR):
    try:
        elements = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((by, selector)))
        return [element.text for element in elements]  # This extracts text from each element found
    except (NoSuchElementException, TimeoutException):
        return []

In [18]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

comments = []
for link in list_links:
    comments.extend(get_commentt(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
df = pd.DataFrame(comments)
df.to_csv('comments_CellPhones.csv', mode='a', header=not os.path.exists('comments_CellPhones.csv'), index=False, encoding='utf-8-sig')
print(df)

https://cellphones.com.vn/iphone-15/review
[{'Platform': 'CellPhones', 'URL': 'https://cellphones.com.vn/iphone-15.html', 'Customer Name': 'Trương Tấn Đô', 'Time Comment': '31/3/2024 04:59', 'Content': 'mong duyệt giúp em khoản góp', 'status': 'Đã mua tại CellphoneS', 'content_rate': 'Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủng\nChất lượng camera Chụp đẹp, chuyên nghiệp\nmong duyệt giúp em khoản góp'}]
[{'Platform': 'CellPhones', 'URL': 'https://cellphones.com.vn/iphone-15.html', 'Customer Name': 'Trương Tấn Đô', 'Time Comment': '31/3/2024 04:59', 'Content': 'mong duyệt giúp em khoản góp', 'status': 'Đã mua tại CellphoneS', 'content_rate': 'Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủng\nChất lượng camera Chụp đẹp, chuyên nghiệp\nmong duyệt giúp em khoản góp'}, {'Platform': 'CellPhones', 'URL': 'https://cellphones.com.vn/iphone-15.html', 'Customer Name': 'Mai Tùng', 'Time Comment': '2/3/2024 09:39', 'Content': 'Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng nên mua máy thôi', 's

In [19]:
# df1 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang web bán hàng\Cellphones\data_19-3\product_smartphone_CellPhones_19-3.csv")
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang web bán hàng\Cellphones\comments_CellPhones.csv")

In [4]:
df1.head()

,Platform,URL,Date Crawled,Product Title,Current Price,Discounted Price,Rating Average,Rating Count
0,CellPhones,https://cellphones.com.vn/iphone-15.html,2024-03-19,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,20.190.000đ,4.9/5,7 đánh giá
1,CellPhones,https://cellphones.com.vn/iphone-15-256gb.html,2024-03-19,iPhone 15 256GB | Chính hãng VN/A,25.990.000đ,23.990.000đ,NaN,NaN
2,CellPhones,https://cellphones.com.vn/iphone-15-512gb.html,2024-03-19,iPhone 15 512GB | Chính hãng VN/A,31.990.000₫,28.490.000₫,NaN,NaN
3,CellPhones,https://cellphones.com.vn/iphone-15-plus.html,2024-03-19,iPhone 15 Plus 128GB | Chính hãng VN/A,25.990.000đ,23.090.000đ,5.0/5,5 đánh giá
4,CellPhones,https://cellphones.com.vn/iphone-15-plus-256gb...,2024-03-19,iPhone 15 Plus 256GB | Chính hãng VN/A,28.990.000đ,26.990.000đ,NaN,NaN


In [5]:
df2.head()

,Platform,URL,Customer Name,Time Comment,Content,content_rate
0,CellPhones,https://cellphones.com.vn/iphone-15.html,Mai Tùng,2/3/2024 09:39,Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
1,CellPhones,https://cellphones.com.vn/iphone-15.html,Ngọc Linh Huyết Kiều (Huyết Kiều Ngọc Linh),7/11/2023 17:04,Tần số quét\n60 Hz\nĐộ sáng tối đa\n2000 nits\...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
2,CellPhones,https://cellphones.com.vn/iphone-15.html,Nguyễn Trần Thụy,7/11/2023 02:19,Awesome phone! Quá tốt,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
3,CellPhones,https://cellphones.com.vn/iphone-15.html,Hiep Nguyen Huy,7/11/2023 02:12,mọi thử ổn trừ camera trước,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
4,CellPhones,https://cellphones.com.vn/iphone-15.html,Nguyễn Trịnh Huy Anh,31/10/2023 21:02,sp tốt nhân viên nhiệt tình,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...


In [6]:
merged_df = pd.merge(df1, df2, on='URL', how='left')

# Hiển thị kết quả
print(merged_df.head())

   Platform_x                                       URL Date Crawled  \
0  CellPhones  https://cellphones.com.vn/iphone-15.html   2024-03-19   
1  CellPhones  https://cellphones.com.vn/iphone-15.html   2024-03-19   
2  CellPhones  https://cellphones.com.vn/iphone-15.html   2024-03-19   
3  CellPhones  https://cellphones.com.vn/iphone-15.html   2024-03-19   
4  CellPhones  https://cellphones.com.vn/iphone-15.html   2024-03-19   

                       Product Title Current Price Discounted Price  \
0  iPhone 15 128GB | Chính hãng VN/A   22.990.000đ      20.190.000đ   
1  iPhone 15 128GB | Chính hãng VN/A   22.990.000đ      20.190.000đ   
2  iPhone 15 128GB | Chính hãng VN/A   22.990.000đ      20.190.000đ   
3  iPhone 15 128GB | Chính hãng VN/A   22.990.000đ      20.190.000đ   
4  iPhone 15 128GB | Chính hãng VN/A   22.990.000đ      20.190.000đ   

  Rating Average Rating Count  Platform_y  \
0          4.9/5   7 đánh giá  CellPhones   
1          4.9/5   7 đánh giá  CellPhones   
2    

In [21]:
merged_df.to_csv('Smarphones_CellPhones_19-3.csv', mode='a', header=not os.path.exists('smarphones_CellPhones_19-3.csv'), index=False, encoding='utf-8-sig')